In [45]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from dateutil.relativedelta import relativedelta, TH

In [46]:
path = pd.DataFrame(glob('BANKNIFTY 2022/*'), columns=['location'])

In [47]:
path

,location
0,BANKNIFTY 2022\bnf_20220103.parquet
1,BANKNIFTY 2022\bnf_20220104.parquet
2,BANKNIFTY 2022\bnf_20220105.parquet
3,BANKNIFTY 2022\bnf_20220106.parquet
4,BANKNIFTY 2022\bnf_20220107.parquet
...,...
243,BANKNIFTY 2022\bnf_20221226.parquet
244,BANKNIFTY 2022\bnf_20221227.parquet
245,BANKNIFTY 2022\bnf_20221228.parquet
246,BANKNIFTY 2022\bnf_20221229.parquet


In [48]:
path['data_date'] = path['location'].apply(lambda x : datetime.datetime.strptime(x.split('_')[-1].split('.')[0], '%Y%m%d'))

In [49]:
path.sort_values(['data_date'], inplace=True)

In [50]:
path.reset_index(drop=True, inplace=True)

In [51]:
path.head()

,location,data_date
0,BANKNIFTY 2022\bnf_20220103.parquet,2022-01-03
1,BANKNIFTY 2022\bnf_20220104.parquet,2022-01-04
2,BANKNIFTY 2022\bnf_20220105.parquet,2022-01-05
3,BANKNIFTY 2022\bnf_20220106.parquet,2022-01-06
4,BANKNIFTY 2022\bnf_20220107.parquet,2022-01-07


In [52]:
for index, row in path.iterrows():
    data = pd.read_parquet(row['location'])
    break

In [53]:
data.head()

,close,date,high,low,oi,open,symbol,time,volume
0,6121.4,2022-01-03,6121.4,6121.4,25.0,6121.4,BANKNIFTY06JAN2230400CE,15:20:00,26
1,3.05,2022-01-03,3.35,2.95,227775.0,3.25,BANKNIFTY06JAN2230400PE,09:15:00,9451
2,2.9,2022-01-03,3.0,2.9,227775.0,3.0,BANKNIFTY06JAN2230400PE,09:16:00,4425
3,2.85,2022-01-03,2.95,2.85,230800.0,2.9,BANKNIFTY06JAN2230400PE,09:17:00,4800
4,2.7,2022-01-03,2.85,2.7,230800.0,2.85,BANKNIFTY06JAN2230400PE,09:18:00,2050


In [54]:
date = row['data_date']

Short Call and Put closest to 0.3 Delta at 9:30 AM annd square off at 3:15 PM. SL of 30% on each leg

In [55]:
entry_time = datetime.datetime.combine(row['data_date'].date(), datetime.time(9,30))

In [56]:
entry_time

datetime.datetime(2022, 1, 3, 9, 30)

In [57]:
exit_time = datetime.datetime.combine(row['data_date'].date(), datetime.time(15,15))

In [58]:
exit_time

datetime.datetime(2022, 1, 3, 15, 15)

In [59]:
trade_log = pd.DataFrame(columns=['Entry_Date' , 'Underlying Price', 'Days_to_Expiry', 'CE_Symbol', 'CE_Entry_Price', 'CE_Exit_price', 'CE_Exit_Datetime', 'PE_Symbol', 'PE_Entry_Price', 'PE_Exit_price', 'PE_Exit_Datetime', 'PnL'])

In [60]:
trade_log

,Entry_Date,Underlying Price,Days_to_Expiry,CE_Symbol,CE_Entry_Price,CE_Exit_price,CE_Exit_Datetime,PE_Symbol,PE_Entry_Price,PE_Exit_price,PE_Exit_Datetime,PnL


In [61]:
underlying_price_open = float(data.loc[(data['symbol'] == 'BANKNIFTY-I') & (data['time'] == '09:15:00'), 'close'].values[0])

In [62]:
underlying_price_open

35811.8516

In [63]:
df_open = data[data['time'] == '09:15:00']

In [64]:
import py_vollib.black_scholes.implied_volatility as iv
import datetime
import numpy as np

call_option_data = df_open[df_open['symbol'].str.endswith('CE')].copy()
call_option_data['time_to_expiration'] = None
call_option_data['implied_volatility'] = 0.2

for index, row in call_option_data.iterrows():
    option_price = float(row['close'])
    stock_price = float(underlying_price_open)
    strike_price = float(row['symbol'].split('CE')[0][-5:])
    expiration_date_str = row['symbol'][9:16]
    expiration_date = datetime.datetime.strptime(expiration_date_str, '%d%b%y').date()
    current_date = datetime.datetime.strptime(row['date'], '%Y-%m-%d').date()
    call_option_data.loc[index, 'time_to_expiration'] = (expiration_date - current_date).days
    time_to_expiration = (expiration_date - current_date).days / 365.0
    risk_free_rate = 0.065  
    initial_guess = 0.2  # Initial estimate of implied volatility

    try:
        implied_volatility = iv.implied_volatility(option_price, stock_price, strike_price, time_to_expiration, risk_free_rate, 'c')
        call_option_data.loc[index, 'implied_volatility'] = implied_volatility
    except:
        continue


put_option_data = df_open[df_open['symbol'].str.endswith('PE')].copy()
put_option_data['time_to_expiration'] = None        
put_option_data['implied_volatility'] = 0.2

for index, row in put_option_data.iterrows():
    option_price = float(row['close'])
    stock_price = float(underlying_price_open)
    strike_price = float(row['symbol'].split('PE')[0][-5:])
    expiration_date_str = row['symbol'][9:16]
    expiration_date = datetime.datetime.strptime(expiration_date_str, '%d%b%y').date()
    current_date = datetime.datetime.strptime(row['date'], '%Y-%m-%d').date()
    put_option_data.loc[index, 'time_to_expiration'] = (expiration_date - current_date).days
    time_to_expiration = (expiration_date - current_date).days / 365.0
    risk_free_rate = 0.065  
    initial_guess = 0.2  # Initial estimate of implied volatility

    try:
        implied_volatility = iv.implied_volatility(option_price, stock_price, strike_price, time_to_expiration, risk_free_rate, 'p')
        put_option_data.loc[index, 'implied_volatility'] = implied_volatility
    except:
        continue



In [65]:
call_option_data

,close,date,high,low,oi,open,symbol,time,volume,time_to_expiration,implied_volatility
6811,2708.95,2022-01-03,2708.95,2708.95,3625.0,2708.95,BANKNIFTY06JAN2233000CE,09:15:00,51,3,0.200000
10640,1720.7,2022-01-03,1734.35,1635.9,40725.0,1635.9,BANKNIFTY06JAN2234000CE,09:15:00,4101,3,0.200000
11630,1537.7,2022-01-03,1537.7,1529.9,2425.0,1529.9,BANKNIFTY06JAN2234200CE,09:15:00,76,3,0.200000
12440,1328.8,2022-01-03,1328.8,1328.8,0.0,1328.8,BANKNIFTY06JAN2234400CE,09:15:00,51,3,0.200000
12872,1256.1,2022-01-03,1267.95,1180.45,56750.0,1201.0,BANKNIFTY06JAN2234500CE,09:15:00,1226,3,0.200000
...,...,...,...,...,...,...,...,...,...,...,...
42087,1.95,2022-01-03,2.1,1.45,24350.0,1.8,BANKNIFTY06JAN2242000CE,09:15:00,8352,3,0.635170
42285,1.85,2022-01-03,2.05,1.5,5900.0,1.5,BANKNIFTY06JAN2242500CE,09:15:00,4176,3,0.673874
42430,1.9,2022-01-03,2.05,1.65,41400.0,2.0,BANKNIFTY06JAN2243000CE,09:15:00,12452,3,0.716913
42692,1.9,2022-01-03,2.0,1.65,69250.0,1.8,BANKNIFTY06JAN2243500CE,09:15:00,8651,3,0.757487


In [66]:
put_option_data

,close,date,high,low,oi,open,symbol,time,volume,time_to_expiration,implied_volatility
1,3.05,2022-01-03,3.35,2.95,227775.0,3.25,BANKNIFTY06JAN2230400PE,09:15:00,9451,3,0.697291
359,3.05,2022-01-03,4.4,2.95,162400.0,4.3,BANKNIFTY06JAN2230500PE,09:15:00,5152,3,0.684727
716,4.15,2022-01-03,4.15,4.1,250.0,4.1,BANKNIFTY06JAN2230700PE,09:15:00,126,3,0.682737
1019,3.3,2022-01-03,4.35,3.15,200825.0,4.25,BANKNIFTY06JAN2231000PE,09:15:00,8976,3,0.627677
1389,3.45,2022-01-03,3.45,3.4,7275.0,3.4,BANKNIFTY06JAN2231100PE,09:15:00,51,3,0.618222
...,...,...,...,...,...,...,...,...,...,...,...
27508,869.1,2022-01-03,928.3,863.0,79275.0,913.0,BANKNIFTY06JAN2236500PE,09:15:00,4602,3,0.351456
28260,952.5,2022-01-03,961.75,945.5,2450.0,960.8,BANKNIFTY06JAN2236600PE,09:15:00,2602,3,0.360536
28985,1049.5,2022-01-03,1049.5,1043.1,1325.0,1043.1,BANKNIFTY06JAN2236700PE,09:15:00,51,3,0.381196
29644,1137.6,2022-01-03,1137.6,1128.85,2125.0,1128.85,BANKNIFTY06JAN2236800PE,09:15:00,151,3,0.392198


In [67]:
import py_vollib.black_scholes.greeks.analytical as greeks
from datetime import datetime as dt

# Calculate delta 
for index, row in call_option_data.iterrows():
    option_price = float(row['close'])
    stock_price = float(row['close'])
    strike_price = float(row['symbol'].split('CE')[0][-5:])
    time_to_expiration = row['time_to_expiration'] / 365.0
    risk_free_rate = 0.065
    volatility = row['implied_volatility']
    
    delta = greeks.delta('c', underlying_price_open, strike_price, time_to_expiration, risk_free_rate, volatility)
    delta_rounded = round(delta, 3)
    call_option_data.loc[index, 'delta'] = abs(delta_rounded)


for index, row in put_option_data.iterrows():
    option_price = row['close']
    stock_price = float(row['close'])
    strike_price = float(row['symbol'].split('PE')[0][-5:])
    time_to_expiration = row['time_to_expiration'] / 365.0
    risk_free_rate = 0.065
    volatility = row['implied_volatility']
    
    delta = greeks.delta('p', underlying_price_open, strike_price, time_to_expiration, risk_free_rate, volatility)
    delta_rounded = round(delta, 3)
    put_option_data.loc[index, 'delta'] = abs(delta_rounded)

In [68]:
call_option_data

,close,date,high,low,oi,open,symbol,time,volume,time_to_expiration,implied_volatility,delta
6811,2708.95,2022-01-03,2708.95,2708.95,3625.0,2708.95,BANKNIFTY06JAN2233000CE,09:15:00,51,3,0.200000,1.000
10640,1720.7,2022-01-03,1734.35,1635.9,40725.0,1635.9,BANKNIFTY06JAN2234000CE,09:15:00,4101,3,0.200000,0.998
11630,1537.7,2022-01-03,1537.7,1529.9,2425.0,1529.9,BANKNIFTY06JAN2234200CE,09:15:00,76,3,0.200000,0.995
12440,1328.8,2022-01-03,1328.8,1328.8,0.0,1328.8,BANKNIFTY06JAN2234400CE,09:15:00,51,3,0.200000,0.988
12872,1256.1,2022-01-03,1267.95,1180.45,56750.0,1201.0,BANKNIFTY06JAN2234500CE,09:15:00,1226,3,0.200000,0.982
...,...,...,...,...,...,...,...,...,...,...,...,...
42087,1.95,2022-01-03,2.1,1.45,24350.0,1.8,BANKNIFTY06JAN2242000CE,09:15:00,8352,3,0.635170,0.003
42285,1.85,2022-01-03,2.05,1.5,5900.0,1.5,BANKNIFTY06JAN2242500CE,09:15:00,4176,3,0.673874,0.003
42430,1.9,2022-01-03,2.05,1.65,41400.0,2.0,BANKNIFTY06JAN2243000CE,09:15:00,12452,3,0.716913,0.003
42692,1.9,2022-01-03,2.0,1.65,69250.0,1.8,BANKNIFTY06JAN2243500CE,09:15:00,8651,3,0.757487,0.003


In [69]:
put_option_data

,close,date,high,low,oi,open,symbol,time,volume,time_to_expiration,implied_volatility,delta
1,3.05,2022-01-03,3.35,2.95,227775.0,3.25,BANKNIFTY06JAN2230400PE,09:15:00,9451,3,0.697291,0.004
359,3.05,2022-01-03,4.4,2.95,162400.0,4.3,BANKNIFTY06JAN2230500PE,09:15:00,5152,3,0.684727,0.004
716,4.15,2022-01-03,4.15,4.1,250.0,4.1,BANKNIFTY06JAN2230700PE,09:15:00,126,3,0.682737,0.006
1019,3.3,2022-01-03,4.35,3.15,200825.0,4.25,BANKNIFTY06JAN2231000PE,09:15:00,8976,3,0.627677,0.005
1389,3.45,2022-01-03,3.45,3.4,7275.0,3.4,BANKNIFTY06JAN2231100PE,09:15:00,51,3,0.618222,0.005
...,...,...,...,...,...,...,...,...,...,...,...,...
27508,869.1,2022-01-03,928.3,863.0,79275.0,913.0,BANKNIFTY06JAN2236500PE,09:15:00,4602,3,0.351456,0.714
28260,952.5,2022-01-03,961.75,945.5,2450.0,960.8,BANKNIFTY06JAN2236600PE,09:15:00,2602,3,0.360536,0.737
28985,1049.5,2022-01-03,1049.5,1043.1,1325.0,1043.1,BANKNIFTY06JAN2236700PE,09:15:00,51,3,0.381196,0.751
29644,1137.6,2022-01-03,1137.6,1128.85,2125.0,1128.85,BANKNIFTY06JAN2236800PE,09:15:00,151,3,0.392198,0.768


In [70]:
# Calculate the absolute difference between delta and 0.3 for call options
call_option_data['delta_diff'] = abs(call_option_data['delta'] - 0.3)

# Calculate the absolute difference between delta and 0.3 for put options
put_option_data['delta_diff'] = abs(put_option_data['delta'] - 0.3)

# Sort the call_option_data dataframe based on delta difference
call_option_data = call_option_data.sort_values('delta_diff')

# Sort the put_option_data dataframe based on delta difference
put_option_data = put_option_data.sort_values('delta_diff')


In [71]:
# Select the row with the minimum delta difference for call options
nearest_call_row = call_option_data.iloc[0]

# Select the row with the minimum delta difference for put options
nearest_put_row = put_option_data.iloc[0]

In [72]:
nearest_call_row

close                                   138.4
date                               2022-01-03
high                                   148.35
low                                     101.0
oi                                   382400.0
open                                    128.0
symbol                BANKNIFTY06JAN2236200CE
time                                 09:15:00
volume                                 104753
time_to_expiration                          3
implied_volatility                   0.219483
delta                                   0.307
delta_diff                              0.007
Name: 24878, dtype: object

In [73]:
nearest_put_row

close                                   190.3
date                               2022-01-03
high                                    242.0
low                                     183.6
oi                                   524075.0
open                                    242.0
symbol                BANKNIFTY06JAN2235300PE
time                                 09:15:00
volume                                 116076
time_to_expiration                          3
implied_volatility                   0.311943
delta                                   0.294
delta_diff                              0.006
Name: 18490, dtype: object

In [74]:
ce_entry_price = float(nearest_call_row['close'])
pe_entry_price = float(nearest_put_row['close'])
days_to_expiry = nearest_call_row['time_to_expiration']
ce_symbol = nearest_call_row['symbol']
pe_symbol = nearest_put_row['symbol']

ce_data = data[data['symbol'] == ce_symbol].copy()
pe_data = data[data['symbol'] == pe_symbol].copy()

ce_data['pos'] = None
pe_data['pos'] = None

ce_sl_price = 1.3*float(ce_entry_price)
pe_sl_price = 1.3*float(pe_entry_price)

In [75]:
lot_size = 25
# Function to calculate P&L for a position
def calculate_pnl(entry_price, exit_price):
    return (exit_price - entry_price)*-1*lot_size

ce_exit_price = 0
pe_exit_price = 0
ce_pnl = 0
pe_pnl = 0

# Iterate through each row in ce_data
for index, row in ce_data.iterrows():
    close_price = float(row['close'])
    time = row['time']
    
    # Check if the close price exceeds the stop-loss price or it's 3:15 PM
    if close_price >= ce_sl_price or time == '15:15:00':
        # Square-off the position (booking a loss)
        ce_data.loc[index, 'pos'] = 0
        ce_exit_price = close_price
        ce_pnl = calculate_pnl(ce_entry_price, ce_exit_price)
        break

# Iterate through each row in pe_data
for index, row in pe_data.iterrows():
    close_price = float(row['close'])
    time = row['time']
    
    # Check if the close price exceeds the stop-loss price or it's 3:15 PM
    if close_price >= pe_sl_price or time == '15:15:00':
        # Square-off the position (booking a loss)
        pe_data.loc[index, 'pos'] = 0
        pe_exit_price = close_price
        pe_pnl = calculate_pnl(pe_entry_price, pe_exit_price)
        break

In [76]:
ce_data[ce_data['pos']==0]

,close,date,high,low,oi,open,symbol,time,volume,pos
24953,181.15,2022-01-03,188.0,175.45,543675.0,176.2,BANKNIFTY06JAN2236200CE,10:30:00,115025,0


In [77]:
pe_data[pe_data['pos']==0]

,close,date,high,low,oi,open,symbol,time,volume,pos
18850,48.7,2022-01-03,48.7,47.2,664875.0,47.25,BANKNIFTY06JAN2235300PE,15:15:00,50075,0


In [78]:
pnl = ce_pnl + pe_pnl

In [79]:
pnl

2471.2500000000005

In [80]:
ce_exit_time = ce_data[ce_data['pos']==0]['time']

In [81]:
pe_exit_time = pe_data[pe_data['pos']==0]['time']

In [82]:
print(ce_exit_time, pe_exit_time)

24953    10:30:00
Name: time, dtype: object 18850    15:15:00
Name: time, dtype: object


In [83]:
ce_pnl

-1068.75

In [84]:
pe_pnl

3540.0000000000005

In [85]:
ce_entry_price

138.4

In [88]:
(pe_entry_price+ce_entry_price)*25

8217.500000000002

In [111]:
time_to_expiration = call_option_data.iloc[0]['time_to_expiration']

result = [date , underlying_price_open, time_to_expiration, ce_symbol, ce_entry_price, ce_exit_price, ce_exit_time, pe_symbol, pe_entry_price, pe_exit_price, pe_exit_time, pnl]

In [112]:


result = [
    pd.to_datetime(result[0]),         
    float(result[1]),                  
    int(result[2]),                    
    result[3],
    float(result[4]),                  
    float(result[5]),                  
    result[6].values[0],        
    result[7],
    float(result[8]),                  
    float(result[9]),                  
    result[10].values[0],        
    float(result[11])                  
]

trade_log.loc[len(trade_log)] = result

In [113]:
trade_log

,Entry_Date,Underlying Price,Days_to_Expiry,CE_Symbol,CE_Entry_Price,CE_Exit_price,CE_Exit_Datetime,PE_Symbol,PE_Entry_Price,PE_Exit_price,PE_Exit_Datetime,PnL
0,2022-01-03,35811.8516,3,BANKNIFTY06JAN2236200CE,138.4,181.15,10:30:00,BANKNIFTY06JAN2235300PE,190.3,48.7,15:15:00,2471.25
